In [2]:
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, QAOA, SamplingVQE
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit.library import TwoLocal
from qiskit.result import QuasiDistribution
from qiskit_aer.primitives import Sampler
from qiskit_finance.applications.optimization import PortfolioOptimization
from qiskit_finance.data_providers import RandomDataProvider
from qiskit_optimization.algorithms import MinimumEigenOptimizer
import numpy as np
import matplotlib.pyplot as plt
import datetime

import pandas as pd
import random
import yfinance as yf
import time

ImportError: cannot import name 'BaseBackend' from 'qiskit.providers' (/Users/vibhu/PycharmProjects/test/venv/lib/python3.10/site-packages/qiskit/providers/__init__.py)

In [ ]:
start_date = "2017-12-25"
end_date = "2022-12-25"

assets = ['AAPL','GS', 'JPM', 'MSFT', 'V']

In [ ]:
stocks_data = yf.download(assets, start = start_date, end = end_date)['Adj Close']  


stocks_return = stocks_data.pct_change()
stocks_return.dropna(inplace=True)
mu = stocks_return.mean()      #Expected Returns
sigma = stocks_return.cov()    #Risk
sigma = np.array(sigma)
mu = np.array(mu)
num_assets = len(mu)

stocks_return

In [ ]:
q = 0.5  # set risk factor
budget = num_assets // 2  # set budget
penalty = num_assets  # set parameter to scale the budget penalty term

portfolio = PortfolioOptimization(
    expected_returns=mu, covariances=sigma, risk_factor=q, budget=budget
)
qp = portfolio.to_quadratic_program()
qp

In [ ]:
from qiskit_optimization.converters import LinearEqualityToPenalty

#removing the constraint to create the QUBO
lineq2penalty = LinearEqualityToPenalty(penalty) 
qubo = lineq2penalty.convert(qp)

model = qubo

#converting QUBO to an Ising Hamiltonian
hamiltonian, offset = qubo.to_ising()


In [ ]:
def print_result(result):
    selection = result.x
    value = result.fval
    print("eigen valuue is ", value)
    print("Optimal: selection {}, value {:.4f}".format(selection, value))

    eigenstate = result.min_eigen_solver_result.eigenstate
    print("eigenstate is ", eigenstate)
    probabilities = (
        eigenstate.binary_probabilities()
        if isinstance(eigenstate, QuasiDistribution)
        else {k: np.abs(v) ** 2 for k, v in eigenstate.to_dict().items()}
    )
    print("\n----------------- Full result ---------------------")
    print("selection\tvalue\t\tprobability")
    print("---------------------------------------------------")
    probabilities = sorted(probabilities.items(), key=lambda x: x[1], reverse=True)

    for k, v in probabilities:
        x = np.array([int(i) for i in list(reversed(k))])
        value = portfolio.to_quadratic_program().objective.evaluate(x)
        print("%10s\t%.4f\t\t%.4f" % (x, value, v))

In [ ]:
# create an instance of the NumPyMinimumEigensolver class,
#which is a classical algorithm for solving eigenvalue problems.
exact_mes = NumPyMinimumEigensolver()

# create an instance of the MinimumEigenOptimizer class, which is used to solve an optimization problem by
# finding its minimum eigenvalue.
# The solver used in this case is the exact_mes solver created in the previous step.
exact_eigensolver = MinimumEigenOptimizer(exact_mes)

# Solve the optimization problem defined by the quadratic program qp using the exact_mes solver
result = exact_eigensolver.solve(qp)

# print the result of the optimization problem using the print_result function defined earlier.
print_result(result)

In [ ]:
# AerSimulator.get_backend
# provider = IBMQ.load_account()

from qiskit import IBMQ

IBMQ.save_account('1ddfa37d28b3fce2197adacfda4217a7c2109b76ca66837262506e4f1944f0497c0651b9b1df1be4c707be43c246d4771993e53ce23d39d608f4b208ba8b636c', overwrite=True)
provider = IBMQ.load_account()
# backend = provider.get_backend('ibm_oslo')
provider

In [ ]:
# AerSimulator.get_backend
# provider = IBMQ.load_account()

# from qiskit import IBMQ

# IBMQ.save_account('ee0757e41299179fb8fd3aab7236e79a12eda6a41c2e860c72fcb9a16274068b3f7c86a6b03428677bd874cfd2a41d90794800294648da41d0141a24dce55ce8', overwrite=True)
# provider = IBMQ.load_account()
# # backend = provider.get_backend('ibm_oslo')
# provider



from qiskit_aer import AerSimulator, AerProvider
from qiskit_aer.noise import NoiseModel
# provider = IBMQ.load_account()


provider = IBMQ.get_provider(group="open")  # replace by your runtime provider
backend = provider.get_backend("ibmq_belem")  # select a backend that supports the runtime
# backend = provider.get_backend("ibmq_qasm_simulator")  # select a backend that supports the runtime
# backend = provider.get_backend('ibmq_qasm_simulator')
# backend = AerSimulator.from_backend(backend)
# backend = AerSimulator.from_backend(backend)

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Session, Options

# service = QiskitRuntimeService(channel="ibm_quantum")
# backend = service.backend("ibmq_qasm_simulator")

cobyla = COBYLA()
cobyla.set_options(maxiter=25)
ry = TwoLocal(num_assets, "ry", "cz", reps=3, entanglement="full")
initial_point = np.random.random(ry.num_parameters)

In [ ]:
# Error on real backend
# Deprecation issues
from qiskit_nature.runtime import VQEClient

runtime_vqe = VQEClient(
    ansatz=ry,
    optimizer=cobyla,
    initial_point=initial_point,
    provider=provider,
    backend=backend,
    shots=1024,
    measurement_error_mitigation=True,
)  # use a complete measurement fitter for error mitigation

In [ ]:
# algo = runtime_vqe.compute_minimum_eigenvalue(hamiltonian)
# # result = algo.solve(qp)
# algo

In [ ]:
# print(algo)

In [ ]:
# selection = result.x
# value = result.fval
# print("eigen valuue is ", value)
# print("Optimal: selection {}, value {:.4f}".format(selection, value))

In [ ]:
# eigenstate = algo.eigenstate
# print("eigenstate is ", eigenstate)
# probabilities = (
#     eigenstate.binary_probabilities()
#     if isinstance(eigenstate, QuasiDistribution)
#     else {k: np.abs(v) ** 2 for k, v in eigenstate.items()}
# )

# probabilities = sorted(probabilities.items(), key=lambda x: x[1], reverse=True)
# print(probabilities)


# print("\n----------------- Full result ---------------------")
# print("selection\tvalue\t\tprobability")
# print("---------------------------------------------------") 


# for k, v in probabilities:
#     x = np.array([int(i) for i in list(reversed(k))])
#     value = portfolio.to_quadratic_program().objective.evaluate(x)
#     print("%10s\t%.4f\t\t%.4f" % (x, value, v))

In [ ]:

# result_dict = algo.eigenstate
# print(result_dict)
# result_bitstring = max(result_dict, key = result_dict.get)
# # result_bitstring
# resulting_number = int(result_bitstring, 2)
# resulting_number

In [ ]:
print(ry.depth())

In [ ]:
print(ry.width())

In [ ]:
from qiskit.utils import algorithm_globals

# backend = AerSimulator.from_backend(backend)

algorithm_globals.random_seed = 1234

# Line not working,
# Need to queue it in the actual QPU
backend = provider.get_backend("ibmq_belem")

cobyla = COBYLA()
cobyla.set_options(maxiter=500)
ry = TwoLocal(num_assets, "ry", "cz", reps=3, entanglement="full")

# Pass real backend in the Sampler class
vqe_mes = SamplingVQE(sampler=Sampler(), ansatz=ry, optimizer=cobyla)

# Runs on the Aersimulator(backend) instead of the real backend
# vqe_mes = SamplingVQE(sampler=Sampler(backend_options=backend), ansatz=ry, optimizer=cobyla)

# Possible ways to get the answer -
# 1. Remove Aersimulator from the Sampler class and hardcode it such that it takes real QPU as backend
# 2.

# Analyze these three classes - SamplingVQE, SamplingMinimumEigensolverResult, MinimumEigenOptimizer
# How? - 1. Input, 2. Output

# Understand the MinimumEigenOptimizer and the solve() method
vqe = MinimumEigenOptimizer(vqe_mes)
result = vqe.solve(qp)
# HERE IS THE SOLVE() METHOD

# print("result is ", result)
# print("selected backend is ", backend)
# print_result(result)

In [ ]:
ry.decompose().draw()

In [ ]:
circ_count = ry.decompose().count_ops()
circ_count

In [ ]:
from qiskit.utils import algorithm_globals

algorithm_globals.random_seed = 1234

cobyla = COBYLA()
cobyla.set_options(maxiter=500)
ry = TwoLocal(num_assets, "ry", "cz", reps=3, entanglement="full")
vqe_mes = SamplingVQE(sampler=Sampler(), ansatz=ry, optimizer=cobyla)


# Try to print this and compare these test witsh ---
# If the results are matching  . . . 
# But, significant post-processing will be required

tests = vqe_mes.compute_minimum_eigenvalue(hamiltonian)
print(tests)

# SamplingMinimumEigensolverResult:

# Read bitstring right to left -- 10010

# vqe = MinimumEigenOptimizer(vqe_mes)
# result = vqe.solve(qp)

# print_result(result)

In [ ]:
algorithm_globals.random_seed = 1234

cobyla = COBYLA()
cobyla.set_options(maxiter=250)

# Problem Statement 
# Sampling VQE takes real QPU as backend. 
# shouldn't go through AER simulator, need real QPU. 
# Go through github repo, all connected files 
#

# Value corresponding to the number: quasi distribution
#

qaoa_mes = QAOA(sampler=Sampler(), optimizer=cobyla, reps=3)
qaoa = MinimumEigenOptimizer(qaoa_mes)
result = qaoa.solve(qp)

print_result(result)

In [ ]:
# from qiskit.utils import algorithm_globals

# backend = AerSimulator.from_backend(backend)

# algorithm_globals.random_seed = 1234

# options = Options()
# options.optimization_level = 2
# options.resilience_level = 3


# cobyla = COBYLA()
# cobyla.set_options(maxiter=20)

# estimator = Estimator(session=backend, options = options)

# ry = TwoLocal(num_assets, "ry", "cz", reps=3, entanglement="full")
# vqe_mes = SamplingVQE(sampler=estimator, ansatz=ry, optimizer=cobyla)
# # vqe = MinimumEigenOptimizer(vqe_mes)
# # result = vqe.solve(qp)


# print("result is ", result)
# print("selected backend is ", backend)
# print_result(result)

In [ ]:
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import OptimizerResult
from qiskit_optimization.algorithms import CplexOptimizer

options = Options()
options.optimization_level = 2
options.resilience_level = 1

backend = "simulator_statevector"
estimator = Estimator(session=backend, options=options)

vqe = VQE(
        estimator,
        ry,
        COBYLA(1),
        initial_point=initial_point,
        # callback=log.update  
    )
## VQE ESTIMATOR - - - - Restuls to be compared

result = vqe.compute_minimum_eigenvalue(hamiltonian)
# Gives optimal values of all theta

# Output of the rest - - output of the tests 
# print(f"full result: {result}")
print("\n \n ")

# Problem Statement 
# Sampling VQE takes real QPU as backend. 


# classical_algo = CplexOptimizer(result)
# bitstring = classical_algo.solve(qp)
# Objective
# print(bitstring)

# Not getting the optimal bitstring

In [ ]:
print("Optimal Circuit - ",result.optimal_circuit , "\n")
print("Eigenvalue -  ", result.eigenvalue, "\n")
print("Cost function evals - ",result.cost_function_evals,"\n")
print("Optimal Point - ",result.optimal_point, "\n")
print("Optimizer Time - ",result.optimizer_time, "\n")
print("Aux Operators Evals - ", result.aux_operators_evaluated, "\n")
print("Optimizer Result - ",result.optimizer_result, "\n")

In [ ]:
from quantum_portfolio_optimization_main.src.solver.vqe.vqe_solver import VQESolver


from qiskit import BasicAer
from qiskit.utils import QuantumInstance
from qiskit.circuit.library import TwoLocal
from qiskit.algorithms.optimizers import SLSQP

cov = stocks_return.cov()

seed = 123


vqe = VQESolver()
vqe.qp(Cov = cov, mu = mu)
vqe.to_ising(Nq = hamiltonian)

# Prepare QuantumInstance
qi = QuantumInstance(backend, seed_transpiler=seed, seed_simulator=seed)

# Select the VQE parameters
# N = cov.shape[0]
ansatz = TwoLocal(num_qubits= hamiltonian.num_qubits, 
                    rotation_blocks=['ry','rz'], 
                    entanglement_blocks='cz',
                    reps=3,
                    entanglement='full')
slsqp = SLSQP(maxiter= 100)
vqe.vqe_instance(ansatz=ansatz, optimizer=slsqp, quantum_instance=qi)

res_vqe = vqe.solve()
print(res_vqe)

In [ ]:
probs = result.eigenstate.binary_probabilities()
probs

In [ ]:
from typing import List, Optional, Union, cast

import numpy as np
from qiskit.algorithms.minimum_eigen_solvers import MinimumEigensolver as LegacyMinimumEigensolver
from qiskit.algorithms.minimum_eigen_solvers import (
    MinimumEigensolverResult as LegacyMinimumEigensolverResult,
)
from qiskit.algorithms.minimum_eigensolvers import (
    NumPyMinimumEigensolver,
    NumPyMinimumEigensolverResult,
    SamplingMinimumEigensolver,
    SamplingMinimumEigensolverResult,
    VQE,
)
from qiskit.opflow import OperatorBase, PauliOp, PauliSumOp

# from ..converters.quadratic_program_to_qubo import QuadraticProgramConverter, QuadraticProgramToQubo
# from ..exceptions import QiskitOptimizationError
# from ..problems.quadratic_program import QuadraticProgram, Variable
# from .optimization_algorithm import (
#     OptimizationAlgorithm,
#     OptimizationResult,
#     OptimizationResultStatus,
#     SolutionSample,
# )

from qiskit.

MinimumEigensolver = Union[
    SamplingMinimumEigensolver, NumPyMinimumEigensolver, LegacyMinimumEigensolver
]
MinimumEigensolverResult = Union[
    SamplingMinimumEigensolverResult, NumPyMinimumEigensolverResult, LegacyMinimumEigensolverResult
]

In [ ]:
# This code is part of Qiskit.
#
# (C) Copyright IBM 2020, 2022.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.

"""A wrapper for minimum eigen solvers to be used within the optimization module."""
from typing import List, Optional, Union, cast

import numpy as np
from qiskit.algorithms.minimum_eigen_solvers import MinimumEigensolver as LegacyMinimumEigensolver
from qiskit.algorithms.minimum_eigen_solvers import (
    MinimumEigensolverResult as LegacyMinimumEigensolverResult,
)
from qiskit.algorithms.minimum_eigensolvers import (
    NumPyMinimumEigensolver,
    NumPyMinimumEigensolverResult,
    SamplingMinimumEigensolver,
    SamplingMinimumEigensolverResult,
    VQE,
)
from qiskit.opflow import OperatorBase, PauliOp, PauliSumOp

from ..converters.quadratic_program_to_qubo import QuadraticProgramConverter, QuadraticProgramToQubo
from ..exceptions import QiskitOptimizationError
from ..problems.quadratic_program import QuadraticProgram, Variable
from .optimization_algorithm import (
    OptimizationAlgorithm,
    OptimizationResult,
    OptimizationResultStatus,
    SolutionSample,
)

MinimumEigensolver = Union[
    SamplingMinimumEigensolver, NumPyMinimumEigensolver, LegacyMinimumEigensolver
]
MinimumEigensolverResult = Union[
    SamplingMinimumEigensolverResult, NumPyMinimumEigensolverResult, LegacyMinimumEigensolverResult
]


class MinimumEigenOptimizationResult(OptimizationResult):
    """Minimum Eigen Optimizer Result."""

    def __init__(
        self,
        x: Optional[Union[List[float], np.ndarray]],
        fval: Optional[float],
        variables: List[Variable],
        status: OptimizationResultStatus,
        samples: Optional[List[SolutionSample]] = None,
        min_eigen_solver_result: Optional[MinimumEigensolverResult] = None,
        raw_samples: Optional[List[SolutionSample]] = None,
    ) -> None:
        """
        Args:
            x: the optimal value found by ``MinimumEigensolver``.
            fval: the optimal function value.
            variables: the list of variables of the optimization problem.
            status: the termination status of the optimization algorithm.
            min_eigen_solver_result: the result obtained from the underlying algorithm.
            samples: the x values, the objective function value of the original problem,
                the probability, and the status of sampling.
            raw_samples: the x values of the QUBO, the objective function value of the QUBO,
                and the probability of sampling.
        """
        super().__init__(
            x=x,
            fval=fval,
            variables=variables,
            status=status,
            raw_results=None,
            samples=samples,
        )
        self._min_eigen_solver_result = min_eigen_solver_result
        self._raw_samples = raw_samples

    @property
    def min_eigen_solver_result(self) -> MinimumEigensolverResult:
        """Returns a result object obtained from the instance of :class:`MinimumEigensolver`."""
        return self._min_eigen_solver_result

    @property
    def raw_samples(self) -> Optional[List[SolutionSample]]:
        """Returns the list of raw solution samples of ``MinimumEigensolver``.

        Returns:
            The list of raw solution samples of ``MinimumEigensolver``.
        """
        return self._raw_samples


class MinimumEigenOptimizer(OptimizationAlgorithm):
    """A wrapper for minimum eigen solvers.

    This class provides a wrapper for minimum eigen solvers from Qiskit to be used within
    the optimization module.
    It assumes a problem consisting only of binary or integer variables as well as linear equality
    constraints thereof. It converts such a problem into a Quadratic Unconstrained Binary
    Optimization (QUBO) problem by expanding integer variables into binary variables and by adding
    the linear equality constraints as weighted penalty terms to the objective function. The
    resulting QUBO is then translated into an Ising Hamiltonian whose minimal eigen vector and
    corresponding eigenstate correspond to the optimal solution of the original optimization
    problem. The provided minimum eigen solver is then used to approximate the ground state of the
    Hamiltonian to find a good solution for the optimization problem.

    Examples:
        Outline of how to use this class:

    .. code-block::

        from qiskit.algorithms.minimum_eigensolver import QAOA
        from qiskit_optimization.problems import QuadraticProgram
        from qiskit_optimization.algorithms import MinimumEigenOptimizer
        problem = QuadraticProgram()
        # specify problem here
        # specify minimum eigen solver to be used, e.g., QAOA
        qaoa = QAOA(...)
        optimizer = MinimumEigenOptimizer(qaoa)
        result = optimizer.solve(problem)
    """

    def __init__(
        self,
        min_eigen_solver: MinimumEigensolver,
        penalty: Optional[float] = None,
        converters: Optional[
            Union[QuadraticProgramConverter, List[QuadraticProgramConverter]]
        ] = None,
    ) -> None:
        """
        This initializer takes the minimum eigen solver to be used to approximate the ground state
        of the resulting Hamiltonian as well as a optional penalty factor to scale penalty terms
        representing linear equality constraints. If no penalty factor is provided, a default
        is computed during the algorithm (TODO).

        Args:
            min_eigen_solver: The eigen solver to find the ground state of the Hamiltonian.
            penalty: The penalty factor to be used, or ``None`` for applying a default logic.
            converters: The converters to use for converting a problem into a different form.
                By default, when None is specified, an internally created instance of
                :class:`~qiskit_optimization.converters.QuadraticProgramToQubo` will be used.

        Raises:
            TypeError: If minimum eigensolver has an invalid type.
            TypeError: When one of converters has an invalid type.
            QiskitOptimizationError: When the minimum eigensolver does not return an eigenstate.
        """
        if isinstance(min_eigen_solver, VQE):
            raise TypeError(
                "MinimumEigenOptimizer does not support this VQE. You can use  "
                "qiskit.algorithms.minimum_eigensolvers.SamplingVQE instead."
            )
        if not isinstance(
            min_eigen_solver,
            (SamplingMinimumEigensolver, NumPyMinimumEigensolver, LegacyMinimumEigensolver),
        ):
            raise TypeError(
                "MinimumEigenOptimizer supports "
                "qiskit.algorithms.minimum_eigensolvers.SamplingMinimumEigensolver, "
                "qiskit.algorithms.minimum_eigensolvers.NumPyMinimumEigensolver, and "
                "qiskit.algorithms.minimum_eigen_solvers.MinimumEigensolver. "
                f"But {type(min_eigen_solver)} is given."
            )
        if not min_eigen_solver.supports_aux_operators():
            raise QiskitOptimizationError(
                "Given MinimumEigensolver does not return the eigenstate "
                "and is not supported by the MinimumEigenOptimizer."
            )
        self._min_eigen_solver = min_eigen_solver
        self._penalty = penalty

        self._converters = self._prepare_converters(converters, penalty)

    def get_compatibility_msg(self, problem: QuadraticProgram) -> str:
        """Checks whether a given problem can be solved with this optimizer.

        Checks whether the given problem is compatible, i.e., whether the problem can be converted
        to a QUBO, and otherwise, returns a message explaining the incompatibility.

        Args:
            problem: The optimization problem to check compatibility.

        Returns:
            A message describing the incompatibility.
        """
        return QuadraticProgramToQubo.get_compatibility_msg(problem)

    @property
    def min_eigen_solver(self) -> MinimumEigensolver:
        """Returns the minimum eigensolver."""
        return self._min_eigen_solver

    @min_eigen_solver.setter
    def min_eigen_solver(self, min_eigen_solver: MinimumEigensolver) -> None:
        """Sets the minimum eigensolver."""
        self._min_eigen_solver = min_eigen_solver

    
    def _solve_internal(
        self,
        operator: OperatorBase,
        offset: float,
        converted_problem: QuadraticProgram,
        original_problem: QuadraticProgram,
    ):
        # only try to solve non-empty Ising Hamiltonians
        eigen_result: Optional[MinimumEigensolverResult] = None
        if operator.num_qubits > 0:
            # NumPyEigensolver does not accept PauliOp but PauliSumOp
            if isinstance(operator, PauliOp):
                operator = PauliSumOp.from_list([(operator.primitive.to_label(), operator.coeff)])
            # approximate ground state of operator using min eigen solver
            eigen_result = self._min_eigen_solver.compute_minimum_eigenvalue(operator)
            print("eigenresult ",eigen_result)
            # analyze results
            raw_samples = None
            if eigen_result.eigenstate is not None:
                raw_samples = self._eigenvector_to_solutions(
                    eigen_result.eigenstate, converted_problem
                )
                raw_samples.sort(key=lambda x: x.fval)
        else:
            # if Hamiltonian is empty, then the objective function is constant to the offset
            x = np.zeros(converted_problem.get_num_binary_vars())
            fval = offset
            raw_samples = [SolutionSample(x, fval, 1.0, OptimizationResultStatus.SUCCESS)]

        if raw_samples is None:
            # if not function value is given, then something went wrong, e.g., a
            # NumPyMinimumEigensolver has been configured with an infeasible filter criterion.
            return MinimumEigenOptimizationResult(
                x=None,
                fval=None,
                variables=original_problem.variables,
                status=OptimizationResultStatus.FAILURE,
                samples=None,
                raw_samples=None,
                min_eigen_solver_result=eigen_result,
            )

        # translate result back to integers and eventually maximization
        samples, best_raw = self._interpret_samples(original_problem, raw_samples, self._converters)
        return cast(
            MinimumEigenOptimizationResult,
            self._interpret(
                x=best_raw.x,
                converters=self._converters,
                problem=original_problem,
                result_class=MinimumEigenOptimizationResult,
                samples=samples,
                raw_samples=raw_samples,
                min_eigen_solver_result=eigen_result,
            ),
        )


In [ ]:
def solve(self, problem: QuadraticProgram) -> MinimumEigenOptimizationResult:
        """Tries to solves the given problem using the optimizer.

        Runs the optimizer to try to solve the optimization problem.

        Args:
            problem: The problem to be solved.

        Returns:
            The result of the optimizer applied to the problem.

        Raises:
            QiskitOptimizationError: If problem not compatible.
        """
        self._verify_compatibility(problem)

        # convert problem to QUBO minimization problem
        problem_ = self._convert(problem, self._converters)

        # construct operator and offset
        operator, offset = problem_.to_ising()

        return self._solve_internal(operator, offset, problem_, problem)


In [ ]:

@property
def min_eigen_solver_result(self) -> MinimumEigensolverResult:
    """Returns a result object obtained from the instance of :class:`MinimumEigensolver`."""
    return self._min_eigen_solver_result

@property
def raw_samples(self) -> Optional[List[SolutionSample]]:
    """Returns the list of raw solution samples of ``MinimumEigensolver``.

    Returns:
        The list of raw solution samples of ``MinimumEigensolver``.
    """
    return self._raw_samples


In [ ]:
eigen_result = min_eigen_solver_result.compute_minimum_eigenvalue(operator)
print("eigenresult ",eigen_result)

In [ ]:
opt_result = my_minimizer(qp, initial_point)
print(opt_result)

In [ ]:
init_guess = np.random.dirichlet(np.ones(num_assets),size=1)
print(init_guess)

In [ ]:
from scipy.optimize import minimize
# expectation = get_expectation(graph)
res = minimize(qp,
               init_guess,
               method='COBYLA')
res

In [ ]:
opt_result = my_minimizer(qp, init_guess)
opt_result

In [ ]:
opt_result.x

# portfolio diversification 

In [ ]:
# Import requisite modules
import math
import datetime

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# Import Qiskit packages
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, QAOA, SamplingVQE
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit.library import TwoLocal
from qiskit_aer.primitives import Sampler
from qiskit_optimization.algorithms import MinimumEigenOptimizer

# The data providers of stock-market data
from qiskit_finance.data_providers import RandomDataProvider
from qiskit_finance.applications.optimization import PortfolioDiversification

In [ ]:
# Generate a pairwise time-series similarity matrix
seed = 123
# set number of assets (= number of qubits)
num_assets = 4
seed = 123

# Generate expected return and covariance matrix from (random) time-series
stocks = ['AAPL','GS', 'JPM', 'MSFT', 'V']
data = RandomDataProvider(
    tickers=stocks,
    start=datetime.datetime(2016, 1, 1),
    end=datetime.datetime(2016, 1, 30),
    seed=seed,
)
data.run()

rho = data.get_similarity_matrix()
rho

In [ ]:
q = 1  # q less or equal than n
n = num_assets

In [ ]:
import math

class ClassicalOptimizer:
    def __init__(self, rho, n, q):

        self.rho = rho
        self.n = n  # number of inner variables
        self.q = q  # number of required selection

    def compute_allowed_combinations(self):
        f = math.factorial
        return int(f(self.n) / f(self.q) / f(self.n - self.q))

    def cplex_solution(self):

        # refactoring
        rho = self.rho
        n = self.n
        q = self.q

        my_obj = list(rho.reshape(1, n**2)[0]) + [0.0 for x in range(0, n)]
        my_ub = [1 for x in range(0, n**2 + n)]
        my_lb = [0 for x in range(0, n**2 + n)]
        my_ctype = "".join(["I" for x in range(0, n**2 + n)])

        my_rhs = (
            [q]
            + [1 for x in range(0, n)]
            + [0 for x in range(0, n)]
            + [0.1 for x in range(0, n**2)]
        )
        my_sense = (
            "".join(["E" for x in range(0, 1 + n)])
            + "".join(["E" for x in range(0, n)])
            + "".join(["L" for x in range(0, n**2)])
        )

        try:
            my_prob = cplex.Cplex()
            self.populatebyrow(my_prob, my_obj, my_ub, my_lb, my_ctype, my_sense, my_rhs)

            my_prob.solve()

        except CplexError as exc:
            print(exc)
            return

        x = my_prob.solution.get_values()
        x = np.array(x)
        cost = my_prob.solution.get_objective_value()

        return x, cost

    def populatebyrow(self, prob, my_obj, my_ub, my_lb, my_ctype, my_sense, my_rhs):

        n = self.n

        prob.objective.set_sense(prob.objective.sense.minimize)
        prob.variables.add(obj=my_obj, lb=my_lb, ub=my_ub, types=my_ctype)

        prob.set_log_stream(None)
        prob.set_error_stream(None)
        prob.set_warning_stream(None)
        prob.set_results_stream(None)

        rows = []
        col = [x for x in range(n**2, n**2 + n)]
        coef = [1 for x in range(0, n)]
        rows.append([col, coef])

        for ii in range(0, n):
            col = [x for x in range(0 + n * ii, n + n * ii)]
            coef = [1 for x in range(0, n)]

            rows.append([col, coef])

        for ii in range(0, n):
            col = [ii * n + ii, n**2 + ii]
            coef = [1, -1]
            rows.append([col, coef])

        for ii in range(0, n):
            for jj in range(0, n):
                col = [ii * n + jj, n**2 + jj]
                coef = [1, -1]

                rows.append([col, coef])

        prob.linear_constraints.add(lin_expr=rows, senses=my_sense, rhs=my_rhs)

In [ ]:
# Instantiate the classical optimizer class
classical_optimizer = ClassicalOptimizer(rho, n, q)

# Compute the number of feasible solutions:
print("Number of feasible combinations= " + str(classical_optimizer.compute_allowed_combinations()))

# Compute the total number of possible combinations (feasible + unfeasible)
print("Total number of combinations= " + str(2 ** (n * (n + 1))))

In [ ]:
#  Visualize the solution
def visualize_solution(xc, yc, x, C, n, K, title_str):
    plt.figure()
    plt.scatter(xc, yc, s=200)
    for i in range(len(xc)):
        plt.annotate(i, (xc[i] + 0.015, yc[i]), size=16, color="r")

    plt.grid()

    for ii in range(n**2, n**2 + n):

        if x[ii] > 0:
            plt.plot(xc[ii - n**2], yc[ii - n**2], "r*", ms=20)

    for ii in range(0, n**2):

        if x[ii] > 0:
            iy = ii // n
            ix = ii % n
            plt.plot([xc[ix], xc[iy]], [yc[ix], yc[iy]], "C2")

    plt.title(title_str + " cost = " + str(int(C * 100) / 100.0))
    plt.show()


In [ ]:
from qiskit.utils import algorithm_globals


class QuantumOptimizer:
    def __init__(self, rho, n, q):
        self.rho = rho
        self.n = n
        self.q = q
        self.pdf = PortfolioDiversification(similarity_matrix=rho, num_assets=n, num_clusters=q)
        self.qp = self.pdf.to_quadratic_program()

    # Obtains the least eigenvalue of the Hamiltonian classically
    def exact_solution(self):
        exact_mes = NumPyMinimumEigensolver()
        exact_eigensolver = MinimumEigenOptimizer(exact_mes)
        result = exact_eigensolver.solve(self.qp)
        return self.decode_result(result)

    def vqe_solution(self):
        algorithm_globals.random_seed = 100
        cobyla = COBYLA()
        cobyla.set_options(maxiter=250)
        ry = TwoLocal(n, "ry", "cz", reps=5, entanglement="full")
        vqe_mes = SamplingVQE(sampler=Sampler(), ansatz=ry, optimizer=cobyla)
        vqe = MinimumEigenOptimizer(vqe_mes)
        result = vqe.solve(self.qp)
        print("vqe solution is ", result)
        return self.decode_result(result)

    def qaoa_solution(self):
        algorithm_globals.random_seed = 1234
        cobyla = COBYLA()
        cobyla.set_options(maxiter=250)
        qaoa_mes = QAOA(sampler=Sampler(), optimizer=cobyla, reps=3)
        qaoa = MinimumEigenOptimizer(qaoa_mes)
        result = qaoa.solve(self.qp)
        return self.decode_result(result)

    def decode_result(self, result, offset=0):
        quantum_solution = 1 - (result.x)
        ground_level = self.qp.objective.evaluate(result.x)
        return quantum_solution, ground_level


In [ ]:
!pip install qiskit-optimization[cplex]

In [ ]:
quantum_optimizer = QuantumOptimizer(rho, n, q)


In [ ]:
# Check if the binary representation is correct. This requires CPLEX
try:
    import cplex

    # warnings.filterwarnings('ignore')
    quantum_solution, quantum_cost = quantum_optimizer.exact_solution()
    print(quantum_solution, quantum_cost)
    classical_solution, classical_cost = classical_optimizer.cplex_solution()
    print(classical_solution, classical_cost)
    if np.abs(quantum_cost - classical_cost) < 0.01:
        print("Binary formulation is correct")
    else:
        print("Error in the formulation of the Hamiltonian")
except Exception as ex:
    print(ex)

In [ ]:
ground_state, ground_level = quantum_optimizer.exact_solution()
print(ground_state)
classical_cost = 1.000779571614484  # obtained from the CPLEX solution
print("ground level cost is ", ground_level)
try:
    if np.abs(ground_level - classical_cost) < 0.01:
        print("Ising Hamiltonian in Z basis is correct")
    else:
        print("Error in the Ising Hamiltonian formulation")
except Exception as ex:
    print(ex)

In [ ]:
vqe_state, vqe_level = quantum_optimizer.vqe_solution()

print("backend used is ", backend)
print(vqe_state, vqe_level)

try:
    if np.linalg.norm(ground_state - vqe_state) < 0.01:
        print("SamplingVQE produces the same solution as the exact eigensolver.")
    else:
        print(
            "SamplingVQE does not produce the same solution as the exact eigensolver, but that is to be expected."
        )
except Exception as ex:
    print(ex)

In [ ]:
xc, yc = data.get_coordinates()

In [ ]:
visualize_solution(xc, yc, ground_state, ground_level, n, q, "Classical")

In [ ]:
visualize_solution(xc, yc, vqe_state, vqe_level, n, q, "VQE")